In [ ]:
# Install required packages

%pip install -U langgraph langchain_community langchain-anthropic langchain_openai langsmith langgraph-swarm 

In [11]:
# Environment Variable Initialization

import getpass
import os

def _set_if_undefined(var_name: str):
    """
    Set an environment variable if it is not already defined.
    
    Args:
        var_name (str): Name of the environment variable to set.
    """
    if not os.environ.get(var_name):
        # Securely prompt the user for input without echoing it on screen
        os.environ[var_name] = getpass.getpass(f"Please provide your {var_name}: ")

# ---- Environment Variables Required ----

_set_if_undefined("OPENAI_API_KEY")         # API key for OpenAI models
_set_if_undefined("ANTHROPIC_API_KEY")      # API key for OpenAI models
_set_if_undefined("LANGSMITH_TRACING")      # Enable LangSmith tracing ("true" to enable)
_set_if_undefined("LANGSMITH_API_KEY")      # API key for LangSmith platform
_set_if_undefined("OPENAI_MODEL")           # Model name (e.g., "gpt-4.1" "gpt-4o", "gpt-3.5-turbo")
_set_if_undefined("ANTHROPIC_MODEL")        # Model name (e.g., "claude-3-7-sonnet-latest")


In [12]:
# Swarm Example:
# In this setup, each agent can communicate with every other agent (many-to-many connections).
# Agents can decide which agent to call next during their reasoning process.

import os
from typing import Annotated
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm


# ---- Tool Definitions ----

# Define a dummy flight_information_tool
@tool
def flight_information_tool(bookingnumber: Annotated[str, "flight booking number"]):
    """Tool to fetch flight information for a given booking number."""
    # (Simulated API call — replace with real API integration)
    json_data = {
        "flightinfo": {
            "bookingnumber": bookingnumber,
            "departure_airport": "ARN" ,
            "arrival_airport": "DXB",
            "departure_time": "13:40",
            "departure_date": "Tuesday, 20 May 2025",
            "arrival_time": "19:50",
            "arrival_date": "Tuesday, 20 May 2025",
            "departure_gate": "Gate F58",
            "upgrade_availability": "Business Class is available for upgrade.",
            "status": "on-time"
        }
    }
    return json_data

# Load the model name from environment variables
openai_model = os.environ["OPENAI_MODEL"]
# Initialize the LLM (Large Language Model) interface
openai_llm = ChatOpenAI(model=openai_model)


# ---- Agent Definitions ----

# Create the 'Viktor' agent:
# - Expert in flight related queries.
# - Can either solve the query directly or hand it off to Walter if necessary.
viktor  = create_react_agent(
    openai_llm,
    tools=[
        flight_information_tool, 
        create_handoff_tool(agent_name="Walter", description="Transfer to Walter, he can help with luggage related queries")  # Tool to transfer the conversation 
    ],
    prompt=(
        "You are an Airline Support Agent (Viktor) designed to assist customers with flight-related queries. "
        "Your role is to provide accurate and relevant information about flights, gate changes, upgrades, and related services. "
        "You have access to a tool to fetch flight data, use the tool whenever required, do NOT guess or make up an answer."
        "Make sure to provide helpful, friendly responses, even if the information is not available."
        "Please keep going until the user’s query is completely resolved before ending your turn and yielding back to the user."
        "Only terminate your turn when you are sure that the query is answered or you have transferred the query to other agents."
    ),
    name="Viktor",
)



# Define a dummy luggage_support_tool
@tool
def luggage_information_tool(bookingnumber: Annotated[str, "flight booking number"]):
    """Tool to fetch luggage information for a given booking number."""
    # (Simulated API call — replace with real API integration)
    json_data = {
        "luggageinfo": {
            "bookingnumber": bookingnumber,
            "belt": "7" ,
            "limit": "30KGs",
            "lost": False,
            "delayed": False,
            "status": "on-belt"
        }
    }
    return json_data

# Load the model name from environment variables
anthropic_model = os.environ["ANTHROPIC_MODEL"]
# Initialize the LLM (Large Language Model) interface
anthropic_llm = ChatAnthropic(model=anthropic_model)

# Create the 'Walter' agent:
# - Expert in luggage related queries.
# - He can only transfer the query back to Viktor if needed.
walter = create_react_agent(
    anthropic_llm,
    tools=[
        luggage_information_tool,
        create_handoff_tool(agent_name="Viktor", description="Transfer to Viktor, he can help with flight related queries")
    ],
    prompt=(
        "You are an Luggage Support Agent (Walter) designed to assist customers with inquiries regarding luggage, baggage claims, and related services."
        "Your role is to provide accurate and relevant information about luggage limits, baggage belts, lost luggage, and luggage delays."
        "You have access to a tool to fetch luggage data, use the tool whenever required, do NOT guess or make up an answer."
        "Make sure to provide helpful, friendly responses, even if the information is not available."
        "Please keep going until the user’s query is completely resolved before ending your turn and yielding back to the user."
        "Only terminate your turn when you are sure that the query is answered or you have transferred the query to other agents."
    ),
    name="Walter",
)



In [17]:
# ---- Swarm and Workflow Definition ----

# In-memory checkpointing to save intermediate agent states during the conversation
checkpointer = InMemorySaver()
# long-term memory
store = InMemoryStore()
# Create a swarm (multi-agent environment) where agents can call each other
workflow = create_swarm(
    [viktor, walter],  # List of agents
    default_active_agent="Walter"  # Walter starts the conversation
)

# Compile the swarm workflow into an executable app
app = workflow.compile(checkpointer=checkpointer, store=store, name="travel_support")

In [14]:
# ---- Visualization ----
app.get_graph().print_ascii()


        +-----------+     
        | __start__ |     
        +-----------+     
          .         ..    
        ..            .   
       .               .. 
+--------+               .
| Viktor |             .. 
+--------+            .   
          .         ..    
           ..     ..      
             .   .        
          +--------+      
          | Walter |      
          +--------+      


In [18]:
# ---- Stream User Interactions ----

# Configuration for conversation (e.g., thread ID to track session)
config = {"configurable": {"thread_id": "11"}}

# --- Turn 1: User asks a flight related question ---

for chunk in app.stream(
    input={"messages": [{"role": "user", "content": "Whats the departure gate for my flight, booking number is ABC1234"}]},
    config=config,
    debug=True,
    subgraphs=True, 
    stream_mode="debug"
):
    print(chunk)
    print("============================")



[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [{'content': 'Whats the departure gate for my flight, booking '
                          'number is ABC1234',
               'role': 'user'}]}
((), {'type': 'checkpoint', 'timestamp': '2025-05-22T21:20:23.600297+00:00', 'step': -1, 'payload': {'config': {'tags': [], 'metadata': ChainMap({'thread_id': '11'}), 'callbacks': None, 'recursion_limit': 25, 'configurable': {'checkpoint_ns': '', 'thread_id': '11', 'checkpoint_id': '1f037529-2aad-6f22-bfff-6d25cb14d055'}}, 'parent_config': None, 'values': {'messages': []}, 'metadata': {'source': 'input', 'writes': {'__start__': {'messages': [{'role': 'user', 'content': 'Whats the departure gate for my flight, booking number is ABC1234'}]}}, 'step': -1, 'parents': {}, 'thread_id': '11'}, 'next': ['__start__'], 'tasks': [{'id': '8dbd4aad-c0e1-cec7-d88e-1395ca9ca740', 'name': '__start__', 'interrupts': (), 'state': None}]

In [19]:
# --- Turn 2: User asks a luggage related question ---

for chunk in app.stream(
    input={"messages": [{"role": "user", "content": "What's my luggage limit?"}]},
    config=config,
    debug=True,
    subgraphs=True, 
    stream_mode="debug"
):
    print(chunk)
    print("============================")

[3:checkpoint] State at the end of step 3:
{'active_agent': 'Viktor',
 'messages': [HumanMessage(content='Whats the departure gate for my flight, booking number is ABC1234', additional_kwargs={}, response_metadata={}, id='fa4e3396-b8f5-44d9-864c-d09ce6e32d8c'),
              AIMessage(content=[{'text': "I understand you're asking about the departure gate for your flight with booking number ABC1234. As a Luggage Support Agent, I don't have direct access to flight gate information. However, I can help you with any luggage-related questions for your flight.\n\nIf you need information about your departure gate, I'll need to transfer you to Viktor, who specializes in flight-related queries like gate information, departure times, and flight status.", 'type': 'text'}, {'id': 'toolu_018MXVCN5T78eRmATVDdmZuS', 'input': {}, 'name': 'transfer_to_viktor', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_015LLRBEaTGfrAGeVzzmW3wT', 'model': 'claude-3-7-sonnet-20250219', 'sto